In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
##Preprocess data
##drop irrelevent feature
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
##Encoder categorical features
label_Encoder_Gender = LabelEncoder()
data['Gender']= label_Encoder_Gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [5]:
##one hot encode geography column
from sklearn.preprocessing import OneHotEncoder
onehot_encode_geography = OneHotEncoder()
geo_encoder = onehot_encode_geography.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [6]:
onehot_columns = onehot_encode_geography.get_feature_names_out(['Geography'])
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(),columns=onehot_columns)
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [7]:
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
##save encodes into pickle file
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_Encoder_Gender,file)

with open('one_hot_encoder_geography.pkl','wb') as file:
    pickle.dump(onehot_encode_geography,file)

In [9]:
X=data.drop('Exited',axis=1)
y=data['Exited']

In [10]:
y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [11]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [12]:
X_train,X_test

(array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
         -0.57946723, -0.57638802],
        [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
          1.72572313, -0.57638802],
        [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
         -0.57946723,  1.73494238],
        ...,
        [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
         -0.57946723, -0.57638802],
        [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
         -0.57946723, -0.57638802],
        [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
          1.72572313, -0.57638802]]),
 array([[-0.57749609,  0.91324755, -0.6557859 , ..., -0.99850112,
          1.72572313, -0.57638802],
        [-0.29729735,  0.91324755,  0.3900109 , ...,  1.00150113,
         -0.57946723, -0.57638802],
        [-0.52560743, -1.09499335,  0.48508334, ..., -0.99850112,
         -0.57946723,  1.73494238],
        ...,
        [ 0.81311987, -1.09499335,  0.77030065, ...,  

In [13]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

## ANN Implementation

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [15]:
## build our ANN model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ##HL1 connected with input layer
    Dense(32,activation='relu'), ##HL2
    Dense(1,activation='sigmoid') ##output layer
])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
opt=tf.keras.optimizers.Adam(learning_rate=0.01)

In [18]:
##compile the model
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [19]:
## set up tensorboard
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [20]:
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [21]:
##setup early stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [22]:
##train the model
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 2s 3ms/step - loss: 0.4024 - accuracy: 0.8314 - val_loss: 0.3514 - val_accuracy: 0.8520
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3587 - accuracy: 0.8516 - val_loss: 0.3429 - val_accuracy: 0.8575
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3512 - accuracy: 0.8571 - val_loss: 0.3469 - val_accuracy: 0.8485
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3453 - accuracy: 0.8571 - val_loss: 0.3434 - val_accuracy: 0.8595
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3421 - accuracy: 0.8608 - val_loss: 0.3412 - val_accuracy: 0.8610
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3404 - accuracy: 0.8584 - val_loss: 0.3461 - val_accuracy: 0.8555
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3378 - accuracy: 0.8620 - val_loss: 0.3453 - val_accuracy: 0.85

In [23]:
model.save('model.h5')

d:\Data Science\NLP\Krish_Naik\ANN Implementation\MyCode\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [48]:
##Load tensorboard extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [44]:
%tensorboard --logdir logs/fit20250608-082853

Reusing TensorBoard on port 6006 (pid 29800), started 0:01:19 ago. (Use '!kill 29800' to kill it.)

In [ ]:
### Load the pickle file